# FooDB
https://foodb.ca

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [2]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [5]:
DROP TABLE IF EXISTS Food;

CREATE TABLE Crop_Group (
  name VARCHAR(2),
  name_scientific VARCHAR(6),
  description VARCHAR(80),
  public_id
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

# Visualizando as Tabelas

## Grupos e Subgrupos de Cultura (Agrícolas)

Cada tupla é um grupo ou subgrupo. Quando o valor de `CGN` é igual a `CGL`, trata-se da descrição de um grupo, senão de um subgrupo.

| Campo | Descrição |
| --- | --- |
| CGN | código do grupo de cultura (agrícola) |
| CGL | código do subgrupo de cultura (agrícola) |
| Crop_Group_Description | descrição do grupo ou subgrupo de cultura |

In [6]:
SELECT * FROM Crop_Group LIMIT 10;

## Alimentos feitos por Receitas

Cada tupla corresponde ao código e descrição de uma receita e o alimento resultante.

| Campo | Descrição |
| --- | --- |
| Food_Code | código de alimento resultante da receita |
| Food_Abbrev_Desc | descrição abreviada do alimento |
| Food_Desc | descrição detalhada do alimento |

In [7]:
SELECT * FROM Food_Description LIMIT 10;

## Ingredientes (Produtos Alimentares de Base)

Cada tupla é um produto alimentar de base usado em receitas.

| Campo | Descrição | Chave para a Tabela |
| --- | --- | --- |
| CGN | código do grupo de cultura (agrícola) | associado à tabela de Grupos e Subgrupos de Cultura |
| CG_Subgroup | código do subgrupo de cultura (agrícola) | associado à tabela de Grupos e Subgrupos de Cultura |
| FCID_Code | código do ingrediente | |
| FCID_Desc | descrição do ingrediente | |

In [8]:
SELECT * FROM FCID_Description LIMIT 10;

## Receitas - Composição

Cada tupla é um item que é parte de uma receita. Cada receita - todas as tuplas com o mesmo `Food_code`.

| Campo | Descrição | Chave para a Tabela |
| --- | --- | --- |
| Food_Code | código da receita | Alimentos feitos por Receitas |
| Mod_Code | código de receita modificada (0 = sem modificação) | |
| Ingredient_Num | número do ingrediente dentro da receita | |
| FCID_Code | código do ingrediente | Ingredientes (Produtos Alimentares) |
| Cooked_Status | estado e cozimento original do ingrediente | Estado de Cozimento do Produto |
| Food_Form | forma alimentar | Forma do Produto Alimentar |
| Cooking_Method | método de cozimento do ingrediente na receita | Método de Cozimento do Produto na Receita |
| Commodity_Weight | participação (em percentual) do ingrediente no peso da receita | |

In [9]:
SELECT * FROM Recipes LIMIT 10;

## Consumo

Cada tupla é um produto alimentar (ingrediente) ingerido por uma pessoa em um dia. Todas as tuplas com mesmo `SEQN` são ingestões da mesma pessoa no dia 1 ou 2 (`DAYCODE`).

| Campo | Descrição | Chave para a Tabela ou Valor |
| --- | --- | --- |
| SEQN  | id do entrevistado | |
| DRABF | indica se o entrevistado (bebê) foi amamentado em qualquer um dos dois dias de retorno | 1 = sim; 2 = não |
| DAYCODE | Dia do diário alimentar. | 1 = dia 1; 2= dia 2 |
| FCID_Code | código do produto alimentar (usado como ingrediente) | Ingredientes (Produtos Alimentares) |
| Cooked_Status | estado e cozimento original do ingrediente | Estado de Cozimento do Produto |
| Food_Form | forma alimentar | Forma do Produto Alimentar |
| Cooking_Method | método de cozimento do ingrediente na receita | Método de Cozimento do Produto na Receita |
| Intake | ingestão. em gramas | |
| Intake_BW | ingestão. em gramas por quilograma de peso corporal (g/kg pc) | |

In [6]:
SELECT * FROM Intake LIMIT 10;

## Número de Subgrupos Agrícolas

In [27]:
SELECT COUNT(*) FROM Crop_Group G;

83

## Grupos/Subgrupos dos Ingredientes de uma Receita

In [32]:
SELECT R.Food_Code, P.Food_Abbrev_Desc, R.FCID_Code, F.FCID_Desc, R.Ingredient_Num, F.CGN, F.CG_Subgroup
FROM Recipes R, Food_Description P, FCID_Description F
WHERE R.FCID_Code = F.FCID_Code AND R.Food_Code = P.Food_Code
LIMIT 10;

## Subgrupos de Ingredientes de Receitas Agregados

In [30]:
DROP VIEW IF EXISTS Recipe_Subgroups;

CREATE VIEW Recipe_Subgroups AS
SELECT R.Food_Code, F.CG_Subgroup, COUNT(*) AS NUMBER
FROM Recipes R, Food_Description P, FCID_Description F
WHERE R.FCID_Code = F.FCID_Code AND R.Food_Code = P.Food_Code
GROUP BY R.Food_Code, F.CG_Subgroup
ORDER BY R.Food_Code, F.CG_Subgroup;

SELECT * FROM Recipe_Subgroups LIMIT 10;

In [31]:
CALL CSVWRITE('../../../data/food-intake/transposed/recipe-subgroups.csv',
              'SELECT * FROM Recipe_Subgroups');

51944